# Assignment: Explainable Deep Learning

# Class: AIPI 590

# Author: Ramil Mammadov

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QZiK9Hk8XBQPF7JkYlpxYWv0i9gIadDk?usp=sharing)








## Setup & imports
This cell installs and imports everything the notebook needs (PyTorch, TorchVision, Pillow, Matplotlib, and pytorch-grad-cam). It also includes a small fallback to install pytorch-grad-cam from GitHub if PyPI is flaky. Finally, it detects whether a GPU is available and sets device accordingly. Nothing in this step changes the model; it just guarantees a clean, reproducible environment.

In [1]:
!pip -q install -U "jedi>=0.18.0"
!pip -q install -U pip setuptools wheel
!pip -q install -U torch torchvision opencv-python matplotlib Pillow

# pytorch-grad-cam
import sys, subprocess
try:
    import pytorch_grad_cam
except Exception:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                               "pytorch-grad-cam>=1.4.8", "--no-cache-dir"])
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                               "git+https://github.com/jacobgil/pytorch-grad-cam"])

# Importing libraries
import os, random, math, warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn.functional as F
import torchvision
from torchvision.models import resnet50, ResNet50_Weights

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from pytorch_grad_cam import GradCAM, GradCAMPlusPlus, ScoreCAM, EigenCAM, XGradCAM, LayerCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


## Oxford-IIIT Pet loader (adapter)

We download the Oxford-IIIT Pet dataset and wrap it with a tiny adapter so every sample returns a consistent (PIL_image, class_name) pair, regardless of TorchVision version. This avoids version-specific target formats and keeps later code simple. We also pick a small subset (≥5 images) to keep visualization snappy.

In [2]:
root = "./data"
pets = torchvision.datasets.OxfordIIITPet(root=root, download=True)

class PetsAdapter:
    """ds[idx] -> (PIL.Image, human_readable_class_name) across torchvision versions."""
    def __init__(self, ds):
        self.ds = ds
        assert hasattr(self.ds, "classes"), "OxfordIIITPet should expose .classes list"
        self.categories = list(self.ds.classes)
        _, tgt0 = self.ds[0]
        if isinstance(tgt0, (tuple, list)):   self._target_kind = "tuple"
        elif isinstance(tgt0, dict):          self._target_kind = "dict"
        else:                                  self._target_kind = "int"

    def __len__(self):  return len(self.ds)

    def __getitem__(self, idx):
        img, tgt = self.ds[idx]
        if self._target_kind == "tuple":  label_index = int(tgt[0])
        elif self._target_kind == "dict": label_index = int(tgt.get("category", tgt.get("label", 0)))
        else:                             label_index = int(tgt)
        cname = self.ds.classes[label_index]
        return img, cname

ds = PetsAdapter(pets)

# Using a small subset for visualization
random.seed(7)
picked_indices = [i for i in range(min(12, len(ds)))]
print("Total picked:", len(picked_indices))


Total picked: 12


## Pretrained ResNet-50 + transforms

Here we load an ImageNet-pretrained ResNet-50 (no training) and grab its canonical preprocessing pipeline (resize, crop, normalize). We also keep the 1,000 ImageNet class names so we can display human-readable predictions.

In [3]:
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights).to(device).eval()

# Preprocessing pipeline recommended by the weights
preprocess = weights.transforms()

# Human-readable ImageNet classes
imagenet_labels = weights.meta["categories"]
print("Model & transforms ready.")


Model & transforms ready.


## Helper functions

These utilities do the unglamorous but essential work. We convert PIL → model tensors, run a forward pass to get logits/probabilities, and convert normalized tensors back to HWC 8-bit images for overlays (fixing the common CHW/HWC plotting error). We also pick the last conv block (layer4[-1]) as the CAM target and provide a small factory to build different CAM variants without repeating code.

In [4]:
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def pil_to_model_tensor(pil_img):
    """PIL -> normalized tensor (1,3,224,224) on device."""
    return preprocess(pil_img).unsqueeze(0).to(device)

@torch.inference_mode()
def predict_logits(t1x3x224):
    logits = model(t1x3x224)
    probs = F.softmax(logits, dim=1)
    return logits, probs

def topk_from_probs(probs, k=5):
    topk_probs, topk_idx = torch.topk(probs, k, dim=1)
    topk_probs = [float(p) for p in topk_probs.squeeze(0).tolist()]
    topk_idx = topk_idx.squeeze(0).tolist()
    topk_labels = [imagenet_labels[i] for i in topk_idx]
    return list(zip(topk_labels, topk_probs))

def tensor_to_showable_rgb(t):
    """(1,3,224,224) normalized -> (224,224,3) uint8 for display."""
    x = t.detach().squeeze(0).cpu().numpy()
    x = np.transpose(x, (1, 2, 0))
    x = (x * IMAGENET_STD) + IMAGENET_MEAN
    x = np.clip(x, 0.0, 1.0)
    return (x * 255.0).round().astype(np.uint8)

def show_image(img, title=None):
    plt.imshow(img); plt.axis("off")
    if title: plt.title(title)

def pick_target_layers_resnet(m):
    return [m.layer4[-1]]

def build_cam(cam_name, model, target_layers):
    cam_name = cam_name.lower()
    if cam_name == "gradcam":     return GradCAM(model=model, target_layers=target_layers)
    if cam_name == "gradcam++":   return GradCAMPlusPlus(model=model, target_layers=target_layers)
    if cam_name == "eigen-cam":   return EigenCAM(model=model, target_layers=target_layers)
    if cam_name == "score-cam":   return ScoreCAM(model=model, target_layers=target_layers)
    if cam_name == "xgrad-cam":   return XGradCAM(model=model, target_layers=target_layers)
    if cam_name == "layer-cam":   return LayerCAM(model=model, target_layers=target_layers)
    raise ValueError(f"Unknown CAM method: {cam_name}")


## Baseline CAMs (Grad-CAM, Grad-CAM++, Eigen-CAM, + Score-/XGrad-CAM)

For each chosen image we: (1) run the model to get a Top-1 class, (2) compute CAM heatmaps for multiple methods targeting that class, and (3) overlay the heatmaps on the image. We plot one figure per image to avoid subplot overflows and save the overlays to disk for our report.

In [5]:
base_methods = ["GradCAM", "GradCAM++", "Eigen-CAM"]
extra_method = "Score-CAM" if device.type == "cuda" else "XGrad-CAM"
methods = base_methods + [extra_method]
print("Methods:", methods)

target_layers = pick_target_layers_resnet(model)
os.makedirs("outputs", exist_ok=True)

all_results = []

for idx in picked_indices:
    pil_img, pet_class = ds[idx]

    # Forward pass
    inp = pil_to_model_tensor(pil_img)
    logits, probs = predict_logits(inp)
    top1_idx = int(torch.argmax(probs, dim=1).item())
    top1_label = imagenet_labels[top1_idx]
    top5 = topk_from_probs(probs, k=5)

    # Base image for overlays
    show_np_uint8 = tensor_to_showable_rgb(inp)
    base_float = show_np_uint8.astype(np.float32) / 255.0

    # One figure per image
    ncols = 1 + len(methods)
    plt.figure(figsize=(4*ncols, 3))
    plt.subplot(1, ncols, 1)
    show_image(show_np_uint8, title=f"Oxford-Pet: {pet_class}\nPred: {top1_label}")

    targets = [ClassifierOutputTarget(top1_idx)]
    heatmaps = {}

    for j, mname in enumerate(methods, start=2):
        with build_cam(mname, model, target_layers) as cam:
            grayscale_cam = cam(input_tensor=inp, targets=targets)[0]
        heatmaps[mname] = grayscale_cam

        overlay = show_cam_on_image(base_float, grayscale_cam, use_rgb=True)
        plt.subplot(1, ncols, j); show_image(overlay, title=mname)

        Image.fromarray(overlay).save(f"outputs/pet_{idx}_{pet_class}_{mname.replace(' ','_')}.jpg")

    plt.tight_layout(); plt.show()

    all_results.append({
        "pet_class": pet_class,
        "imagenet_top1": top1_label,
        "top5": top5,
        "heatmaps": heatmaps,
    })


Output hidden; open in https://colab.research.google.com to view.

## Logit masking (restrict to cat/dog classes)

Because our model is trained on ImageNet, it can pick classes like “plastic bag.” Logit masking keeps the backbone untouched but re-scores only among ImageNet classes containing “cat” or “dog.” That reduces label-space mismatch and steers predictions (and thus explanations) toward pet-relevant classes without any training.

In [6]:
catdog_indices = [i for i, n in enumerate(imagenet_labels)
                  if ("cat" in n.lower()) or ("dog" in n.lower())]

def predict_with_label_mask(inp, allowed_indices=catdog_indices, topk=5):
    logits = model(inp)
    mask = torch.full_like(logits, -1e9)
    mask[:, allowed_indices] = 0.0
    masked_logits = logits + mask
    local = torch.softmax(masked_logits[:, allowed_indices], dim=1)
    k = min(topk, len(allowed_indices))
    probs_k, idx_local = torch.topk(local, k=k, dim=1)
    idx_local = idx_local.squeeze(0).tolist()
    probs_k = probs_k.squeeze(0).tolist()
    idx_global = [allowed_indices[i] for i in idx_local]
    labels = [imagenet_labels[i] for i in idx_global]
    top1_global_idx = idx_global[0]
    return list(zip(labels, probs_k)), top1_global_idx


## Cat/Dog detector crop (OPTIONAL)

We run a lightweight COCO Faster R-CNN to find cat/dog boxes, then crop the image around the best detection (with a small margin). This reduces background dominance (e.g., bags, blankets) and produces more object-centric inputs before classification. If no cat/dog is detected confidently, we safely fall back to the original image.

In [7]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights

det_weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
detector = fasterrcnn_resnet50_fpn(weights=det_weights).to(device).eval()
det_tfms = det_weights.transforms()
coco_names = det_weights.meta["categories"]
cat_id = coco_names.index("cat")
dog_id = coco_names.index("dog")

def crop_catdog(pil_img, score_thresh=0.6, margin=0.10):
    """Crop around the best cat/dog box; fallback to original image."""
    img_t = det_tfms(pil_img).to(device)
    with torch.no_grad():
        out = detector([img_t])[0]
    boxes = out["boxes"].detach().cpu().numpy()
    labels = out["labels"].detach().cpu().numpy()
    scores = out["scores"].detach().cpu().numpy()

    keep = [i for i,(lab,s) in enumerate(zip(labels, scores))
            if (lab in (cat_id, dog_id)) and (s >= score_thresh)]
    if not keep:
        return pil_img

    i = int(np.argmax(scores[keep]))
    x1, y1, x2, y2 = boxes[keep[i]]
    W, H = pil_img.size
    w, h = x2 - x1, y2 - y1
    x1 = max(0, int(x1 - margin * w)); y1 = max(0, int(y1 - margin * h))
    x2 = min(W, int(x2 + margin * w)); y2 = min(H, int(y2 + margin * h))
    return pil_img.crop((x1, y1, x2, y2))


## Audit loop (Original vs Masked vs Cropped+Masked with Grad-CAM++ & Eigen-CAM) (OPTIONAL)

This is our “sanity-check dashboard.” For a few images, we show: the unmodified prediction and maps; the masked prediction/maps (cat-only); and the cropped+masked prediction/maps. We use Grad-CAM++ (better for multiple salient parts/instances) and Eigen-CAM (good object-level coverage) to compare how each mitigation shifts attention away from backgrounds and toward the animal. It’s a lightweight, repeatable QA step we can run anytime to catch shortcuts and multi-instance issues.

In [8]:
def run_cam(model, inp, class_idx, method="++"):
    target_layers = pick_target_layers_resnet(model)
    Target = ClassifierOutputTarget(class_idx)
    cam_cls = GradCAMPlusPlus if method == "++" else EigenCAM
    with cam_cls(model=model, target_layers=target_layers) as cam:
        return cam(input_tensor=inp, targets=[Target])[0]  # HxW

def audit_image(idx, title_prefix=""):
    pil_img, pet_class = ds[idx]

    # Original (unmasked)
    x = pil_to_model_tensor(pil_img)
    _, p = predict_logits(x)
    top1_full = int(torch.argmax(p,1))
    label_full = imagenet_labels[top1_full]

    # Masked to cat
    top5_masked, top1_masked = predict_with_label_mask(x)
    label_masked = imagenet_labels[top1_masked]

    # Cropped + masked
    crop = crop_catdog(pil_img)
    x_crop = pil_to_model_tensor(crop)
    top5_crop_masked, top1_crop_masked = predict_with_label_mask(x_crop)
    label_crop_masked = imagenet_labels[top1_crop_masked]

    base_orig = tensor_to_showable_rgb(x).astype(np.float32)/255.0
    base_crop = tensor_to_showable_rgb(x_crop).astype(np.float32)/255.0

    campp_orig = run_cam(model, x,      top1_masked, method="++")
    eigen_orig = run_cam(model, x,      top1_masked, method="eigen")
    campp_crop = run_cam(model, x_crop, top1_crop_masked, method="++")
    eigen_crop = run_cam(model, x_crop, top1_crop_masked, method="eigen")

    fig, axes = plt.subplots(2, 3, figsize=(15, 8))

    axes[0,0].imshow(tensor_to_showable_rgb(x)); axes[0,0].axis("off")
    axes[0,0].set_title(f"{title_prefix}Orig\nPred: {label_full}")

    axes[1,0].imshow(crop); axes[1,0].axis("off")
    axes[1,0].set_title(f"Crop\nPred(masked): {label_crop_masked}")

    axes[0,1].imshow(show_cam_on_image(base_orig, campp_orig, use_rgb=True)); axes[0,1].axis("off")
    axes[0,1].set_title("Grad-CAM++ (orig, masked target)")

    axes[1,1].imshow(show_cam_on_image(base_crop, campp_crop, use_rgb=True)); axes[1,1].axis("off")
    axes[1,1].set_title("Grad-CAM++ (crop, masked target)")

    axes[0,2].imshow(show_cam_on_image(base_orig, eigen_orig, use_rgb=True)); axes[0,2].axis("off")
    axes[0,2].set_title("Eigen-CAM (orig, masked target)")

    axes[1,2].imshow(show_cam_on_image(base_crop, eigen_crop, use_rgb=True)); axes[1,2].axis("off")
    axes[1,2].set_title("Eigen-CAM (crop, masked target)")

    plt.tight_layout(); plt.show()

# Run audit on a few images
for k in [0, 1, 2]:
    audit_image(picked_indices[k], title_prefix=f"Oxford-Pet: {ds[picked_indices[k]][1]}\n")


Output hidden; open in https://colab.research.google.com to view.

## Compare & contrast: Grad-CAM vs. its variants

Across all Abyssinian images, the four methods agree on the face as the key region, but they differ in how they highlight it:

- Grad-CAM concentrates on the most discriminative part (often a single eye + whisker pad). It’s sharp and part-centric. In the two-kitten scene it mostly locks onto the front cat and under-covers the second one.

- Grad-CAM++ distributes relevance more evenly across multiple facial parts and across multiple instances when they’re present. In the twin-kitten image it lights both faces better than vanilla Grad-CAM. It’s a bit broader/softer but more complete.

- Eigen-CAM produces a cohesive, object-level mask—a smooth oval over the entire head/upper torso with a bright core. It has the least background leakage when the class is correct, and it’s a great sanity check that the model “sees the whole head,” not just a tiny texture.

- XGrad-CAM tends to mirror Grad-CAM’s choices but with slightly crisper transitions. Where Grad-CAM fires on eye/cheek, XGrad-CAM usually agrees, refining the boundary.

In the “blue shopping bag” photo, before any mitigation the model’s Top-1 is “plastic bag,” and Grad-/XGrad-CAM heat sits on the bag rim and folds. After my label mask (cat/dog only) and detector crop, the Top-1 flips to “Egyptian cat” and both Grad-CAM++ and Eigen-CAM shift decisively to the eyes, muzzle, and ear—a clear demonstration that the mitigations remove a background shortcut and make all maps more semantically aligned.

## Reflection

### What visual cues the model attends to
The classifier relies foremost on facial structure: bright eye rims, the bridge of the nose, the whisker pad/muzzle, and ear triangles. Texture matters too—short, fine fur and high-contrast edges around the face. When the mouth is open, teeth and the dark oral cavity become dominant cues (we saw this in the “cougar” misprediction). In cluttered scenes, strong color/edge patterns near the face (e.g., the blue bag seam, blanket folds) can also attract attention.

### Surprising or misleading behavior
Two issues are especially visible in our panels. First, context dominance: the “plastic bag” Top-1 arises because the bag occupies a large portion of the 224×224 crop and has highly discriminative textures for ImageNet classes; Grad-/XGrad-CAM faithfully highlight the bag, explaining the wrong prediction. Second, label-space mismatch: using an ImageNet head means outputs like “cougar” are possible; the maps (sensibly) light teeth/face for that class, but that’s not the label we care about in a pet-breed setting. There’s also a multi-instance bias: vanilla Grad-CAM often prefers one cat in the two-kitten shot, while Grad-CAM++ spreads attention more fairly. Watermarks and high-contrast logos sometimes draw weak activations, but in our examples they don’t dominate once the label is cat-restricted.

### Why explainability matters here (pet perception)
If this system were used in a home robot, shelter intake, or monitoring context, actions should be driven by the animal, not by props or backgrounds. The CAMs make that verifiable. They also guide practical fixes: our logit masking and cat crop demonstrably pull focus from background objects to the cat’s face, and the audit grid with Grad-CAM++ and Eigen-CAM lets we re-check that behavior after any change. Finally, method choice itself is part of good practice: Grad-CAM++ is preferable in multi-pet scenes; Eigen-CAM gives a robust object-level sanity check; Grad-/XGrad-CAM are great for pinpointing the single most discriminative facial part. Together, these tools help we detect shortcuts early, collect better data (more diverse backgrounds, tighter crops), and build user trust that the model’s decisions come from the right pixels.

## LLM
For this assignment, **OpenAI Chat GPT-5 Thinking** large language model was utilized to generate some of the Python code for the modeling and visualization steps. The tool was also used for correcting and adjusting markdown formatting.